In [ ]:
import pymc3 as pm
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import theano.tensor as tt
#
tbl = pd.read_csv('ratings_anon.csv')
Nrater = len(tbl['rater_id'].unique())
raterID = np.asarray(tbl['rater_id'].astype('category').cat.codes, dtype=np.int16)
raterSex = np.asarray(tbl['rater_sex'].astype('category').cat.codes, dtype=np.int16)
subjID = np.asarray(tbl['face_id'].astype('category').cat.codes, dtype=np.int16)

progesterone = np.asarray(tbl['progesterone'], dtype=np.float64)
estradiol = np.asarray(tbl['estradiol'], dtype=np.float64)
rating = np.asarray(tbl['rating'], dtype=np.float64)
rating = np.log(rating - 0) - np.log(8 - rating)
whr_raw = np.asarray(tbl['whr'], dtype=np.float64)

sortID, index = np.unique(subjID, return_index=True)
index = index[np.isfinite(whr_raw[index])]
whr = np.asarray(whr_raw[index], dtype=np.float64)

# Model 1

In [ ]:
Nmeasure = 4
with pm.Model() as model:
    # on the assumption that attractiveness, progesterone and estradiol 
    # are correlated, this correlation could be capture with a cov matrix
    sd_dist = pm.HalfCauchy.dist(beta=2.5)
    packed_chol = pm.LKJCholeskyCov('chol_cov', n=Nmeasure, eta=1, sd_dist=sd_dist)
    # compute the covariance matrix
    chol = pm.expand_packed_triangular(Nmeasure, packed_chol, lower=True)
    cov = tt.dot(chol, chol.T)
    
    # Extract the correlation deviations etc
    sd = pm.Deterministic('sd',tt.sqrt(tt.diag(cov)))
    corr = tt.diag(sd**-1).dot(cov.dot(tt.diag(sd**-1)))
    r = pm.Deterministic('r', corr[np.triu_indices(Nmeasure, k=1)])
    
    # transform an uncorrelated normal:
    mu0 = pm.Normal('mu0', mu=np.zeros((Nmeasure,1)), sd=1., 
                    shape=(Nmeasure,)+np.unique(subjID).shape)
    musubj = tt.dot(chol, mu0)
    
    subj_sd = pm.HalfCauchy('subj_sd', beta=2.5,
                            shape=(Nmeasure-1,)+np.unique(subjID).shape)
    
    intercept1 = pm.Normal('interp1', mu=np.log(progesterone.mean()), sd=10.)
    sess_beta1 = pm.Normal('sebeta1', mu=0., sd=5., shape=Nsess)
    obs1 = pm.Lognormal('progesterone', 
                        mu=tt.exp(musubj[0, subjID_hm]+sess_beta1[sessID_hm]+intercept1),
                        sd=subj_sd[0, subjID_hm],
                        observed=progesterone)
    
    intercept2 = pm.Normal('interp2', mu=np.log(estradiol.mean()), sd=10.)
    sess_beta2 = pm.Normal('sebeta2', mu=0., sd=5., shape=Nsess)
    obs2 = pm.Lognormal('estradiol', 
                        mu=tt.exp(musubj[1, subjID_hm]+sess_beta2[sessID_hm]+intercept2),
                        sd=subj_sd[1, subjID_hm],
                        observed=estradiol)
    
    ratermu = pm.Normal('ratermu', mu=0., sd=10., shape=np.unique(raterID).shape)
    ratersd = pm.HalfNormal('ratersd', sd=10., shape=np.unique(raterID).shape)
    obs3 = pm.Normal('rating', 
                     mu=musubj[2, subjID]+ratermu[raterID],
                     sd=subj_sd[2, subjID]+ratersd[raterID],
                     observed=rating_r)
    
    intercept3 = pm.Normal('interp3', mu=whr.mean(), sd=1.)
    whr_mu = pm.Deterministic('whr_mu', musubj[3, subjID[index]]+intercept3)
    obs4 = pm.Normal('whr', 
                     mu=whr_mu,
                     sd=.01,
                     observed=whr)
    
    trace = pm.sample(2000, njobs=4, tune=2000)

In [ ]:
pm.traceplot(trace, varnames=['r','chol_cov']);

In [ ]:
CONVERGENCE_TITLE = lambda: 'BFMI = {a:.2f}\nmax(R_hat) = {b:.3f}\nmin(Eff_n) = {c:.3f}'\
                    .format(a=bfmi, b=max_gr, c=min_effn)
def get_diags(trace):
    bfmi = pm.bfmi(trace)
    max_gr = max(np.max(gr_stats) for gr_stats in pm.gelman_rubin(trace).values())
    min_effn = min(np.min(ef_stats) for ef_stats in pm.effective_n(trace).values())
    return bfmi, max_gr, min_effn

_, ax = plt.subplots(1, 1, figsize=(7, 5))
bfmi, max_gr, min_effn = get_diags(trace)
(pm.energyplot(trace, ax=ax)
   .set_title(CONVERGENCE_TITLE()));

In [ ]:
corrpost = np.ones((Nmeasure, Nmeasure))
corrpost[np.triu_indices(Nmeasure, k=1)] = pm.df_summary(trace,varnames=['r'])['mean'].values
corrpost[np.tril_indices(Nmeasure, k=-1)] = pm.df_summary(trace,varnames=['r'])['mean'].values

#replace empty element in whr with mean
whr_raw[~np.isfinite(whr_raw)]=np.nanmean(whr_raw)

corermpr = np.corrcoef(np.vstack([progesterone, estradiol, rating_r, whr_raw]))
tmp = []
for id_ in subjID[np.sort(index)]:
    tmp.append(np.vstack([progesterone[subjID==id_][:5],
                          estradiol[subjID==id_][:5],
                          whr_raw[subjID==id_][:5],
                          [id_]*5]).T)
tmp = np.asarray(tmp)
tmp2 = np.reshape(tmp,[tmp.shape[0]*tmp.shape[1],tmp.shape[2]])
corrtmp = np.corrcoef(tmp2[:,:3].T)
corermpr[0,1]=corermpr[1,0]=corrtmp[0,1]
corermpr[0,3]=corermpr[3,0]=corrtmp[0,2]
corermpr[1,3]=corermpr[3,1]=corrtmp[1,2]

measure_labels = ['Progesterone', 'Estradiol', 'Rating', 'WHR']

_, ax = plt.subplots(1, 2, figsize=(10, 5))
empirical_corr = pd.DataFrame(corermpr,
                              columns=measure_labels,
                              index=measure_labels)
sns.heatmap(empirical_corr, 
            cbar=False, square = True, annot=True, 
            linewidths=.1, cmap='viridis', ax=ax[0])
ax[0].set_title('Empirical Correlation Matrix')

posterior_corr = pd.DataFrame(corrpost,
                              columns=measure_labels,
                              index=measure_labels)
sns.heatmap(posterior_corr, 
            cbar=False, square = True, annot=True, 
            linewidths=.1, cmap='viridis', ax=ax[1])
ax[1].set_title('Latent Correlation Matrix')

plt.tight_layout();

In [ ]:
plotpost = pm.plots.artists.plot_posterior_op
tri_idx = np.triu_indices(Nmeasure, k=1)
_,  ax = plt.subplots(2, 3, figsize=(18, 6), sharex=True)
corrtrace = trace['r']
ax1 = ax.flatten()
for i in range(len(tri_idx[0])):
    trace_values = corrtrace[:, i]
    plotpost(trace_values, ax1[i], kde_plot=False, point_estimate='mean', 
             round_to=3, alpha_level=0.05, ref_val=0., rope=None, color='#87ceeb')
    ax1[i].axvline(corermpr[tri_idx[0][i], tri_idx[1][i]])
    ax1[i].set_title('Corr('+measure_labels[tri_idx[0][i]]+', '+measure_labels[tri_idx[1][i]]+')')
plt.tight_layout();

In [ ]:
ppc = pm.sample_ppc(trace,100, model=model)

In [ ]:
xlim1 = max(progesterone)
_, ax = plt.subplots()
for ippc in ppc['progesterone']:
    #plt.hist(ippc, bins=100, normed=True, histtype='step', color='gray');
    pm.kdeplot(ippc[ippc<xlim1], color='gray', alpha=.5, ax=ax);
pm.kdeplot(progesterone, color='r', ax=ax);
#plt.hist(progesterone, bins=100, normed=True, histtype='step', color='r');
plt.xlim([0,xlim1]);

In [ ]:
xlim1 = max(estradiol)
_, ax = plt.subplots()
for ippc in ppc['estradiol']:
    #plt.hist(ippc, bins=100, normed=True, histtype='step', color='gray');
    pm.kdeplot(ippc[ippc<xlim1], color='gray', alpha=.5, ax=ax);
pm.kdeplot(estradiol, color='r', ax=ax);
#plt.hist(progesterone, bins=100, normed=True, histtype='step', color='r');
plt.xlim([0,xlim1]);

In [ ]:
xlim1 = max(rating_r)
_, ax = plt.subplots()
for ippc in ppc['rating']:
    #plt.hist(ippc, bins=100, normed=True, histtype='step', color='gray');
    pm.kdeplot(ippc[ippc<xlim1], color='gray', alpha=.5, ax=ax);
pm.kdeplot(rating_r, color='r', ax=ax);
#plt.hist(progesterone, bins=100, normed=True, histtype='step', color='r');

In [ ]:
def plot_predi(m):
    bandwidth = 200
    k = round(m*(len(rating_r)-bandwidth))

    idx = range(k, k+bandwidth)
    f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18, 10))
    ax1.plot(ppc['progesterone'][:,idx].T,alpha=.05,color='gray')
    ax1.plot(progesterone[idx],color='r',lw=1)
    ax1.set_ylim([0,1900]);

    ax2.plot(ppc['estradiol'][:,idx].T,alpha=.05,color='gray')
    ax2.plot(estradiol[idx],color='r',lw=1)
    ax2.set_ylim([0,25]);

    ax3.plot(ppc['rating'][:,idx].T,alpha=.05,color='gray')
    ax3.plot(rating_r[idx],color='r',lw=1)

    plt.tight_layout()
    plt.show()

from ipywidgets import interactive

interactive_plot = interactive(plot_predi, m=(0., 1., .01))
interactive_plot

In [ ]:
plt.figure(figsize=(18, 6))
plt.plot(ppc['whr'].T,alpha=.05,color='gray')
plt.plot(whr,color='r',lw=1)
plt.ylim([whr.min(), whr.max()]);

## Model 2

In [ ]:
class Ordered(pm.distributions.transforms.ElemwiseTransform):
    name = "ordered"

    def forward(self, x):
        out = tt.zeros(x.shape)
        out = tt.inc_subtensor(out[0], x[0])
        out = tt.inc_subtensor(out[1:], tt.log(x[1:] - x[:-1]))
        return out
    
    def forward_val(self, x, point=None):
        x, = pm.distributions.distribution.draw_values([x], point=point)
        return self.forward(x)

    def backward(self, y):
        out = tt.zeros(y.shape)
        out = tt.inc_subtensor(out[0], y[0])
        out = tt.inc_subtensor(out[1:], tt.exp(y[1:]))
        return tt.cumsum(out)

    def jacobian_det(self, y):
        return tt.sum(y[1:])

In [ ]:
Nmeasure = 4
Nrating = len(np.unique(rating))
with pm.Model() as model2:
    # on the assumption that attractiveness, progesterone and estradiol 
    # are correlated, this correlation could be capture with a cov matrix
    sd_dist = pm.HalfCauchy.dist(beta=2.5)
    packed_chol = pm.LKJCholeskyCov('chol_cov', n=Nmeasure, eta=1, sd_dist=sd_dist)
    # compute the covariance matrix
    chol = pm.expand_packed_triangular(Nmeasure, packed_chol, lower=True)
    cov = tt.dot(chol, chol.T)
    
    # Extract the correlation deviations etc
    sd = pm.Deterministic('sd',tt.sqrt(tt.diag(cov)))
    corr = tt.diag(sd**-1).dot(cov.dot(tt.diag(sd**-1)))
    r = pm.Deterministic('r', corr[np.triu_indices(Nmeasure, k=1)])
    
    # transform an uncorrelated normal:
    mu0 = pm.Normal('mu0', mu=np.zeros((Nmeasure,1)), sd=1., 
                    shape=(Nmeasure,)+np.unique(subjID).shape)
    musubj = tt.dot(chol, mu0)
    
    subj_sd = pm.HalfCauchy('subj_sd', beta=2.5,
                            shape=(Nmeasure-1,)+np.unique(subjID).shape)
    
    intercept1 = pm.Normal('interp1', mu=np.log(progesterone.mean()), sd=10.)
    obs1 = pm.Lognormal('progesterone', 
                        mu=tt.exp(musubj[0, subjID]+intercept1),
                        sd=subj_sd[0, subjID],
                        observed=progesterone)
    
    intercept2 = pm.Normal('interp2', mu=np.log(estradiol.mean()), sd=10.)
    obs2 = pm.Lognormal('estradiol', 
                        mu=tt.exp(musubj[1, subjID]+intercept2),
                        sd=subj_sd[1, subjID],
                        observed=estradiol)
    
    ratermu = pm.Normal('ratermu', mu=0., sd=10., shape=np.unique(raterID).shape)
    ratersd = pm.HalfNormal('ratersd', sd=10., shape=np.unique(raterID).shape)
    unit_n = pm.Normal('unit_n',  mu=0, sd=1)
    latentrate = musubj[2, subjID]+ratermu[raterID] + subj_sd[2, subjID]+ratersd[raterID]*unit_n
    # latent rating
    a = pm.Normal('a', 0., 10., 
                  transform=Ordered(), 
                  shape=Nrating, 
                  testval=np.arange(Nrating) - Nrating/2)
    pa = pm.math.sigmoid(tt.shape_padleft(a) - tt.shape_padright(latentrate))
    p_cum = tt.concatenate([
                            tt.zeros_like(tt.shape_padright(pa[:, 0])),
                            pa,
                            tt.ones_like(tt.shape_padright(pa[:, 0]))
                            ], axis=1)
    p = p_cum[:, 1:] - p_cum[:, :-1]
    obs3 = pm.Categorical('rating', p, observed=rating - 1)
    
    intercept3 = pm.Normal('interp3', mu=whr.mean(), sd=1.)
    whr_mu = pm.Deterministic('whr_mu', musubj[3, subjID[index]]+intercept3)
    obs4 = pm.Normal('whr', 
                     mu=whr_mu,
                     sd=.01, # measurement error
                     observed=whr)

    trace2 = pm.sample(2000, njobs=4, tune=2000)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(7, 5))
bfmi, max_gr, min_effn = get_diags(trace2)
(pm.energyplot(trace2, ax=ax)
   .set_title(CONVERGENCE_TITLE()));

In [ ]:
MODEL_NAME_MAP = {
    0: "model1",
    1: "model2"
}

comp_df = (pm.compare([trace, trace2], 
                      [model, model2])
             .rename(index=MODEL_NAME_MAP)
             .loc[MODEL_NAME_MAP.values()])

comp_df